In [1]:
import pandas as pd
import pandas_datareader.data as web
import statsmodels.api as sm
import os
import numpy as np


# List files in the current directory
os.listdir()

# 1. Fetch UMCSENT data from FRED
from fredapi import Fred
fred = Fred(api_key='5e450d83c52ef0ad9cca37fad13d8fab')

temp = fred.get_series('UMCSENT', observation_start='1978-01-01', observation_end='2023-07-01')
df_sentiment = pd.DataFrame(data = temp, columns=['consumer_sentiment_index'])
df_sentiment = df_sentiment.rename_axis('Date').reset_index()
print(df_sentiment.head(5))

# OECD Total share price index 
#temp = fred.get_series('SPASTT01USM661N', observation_start='1960-01-01', observation_end='2023-07-01')
#df_total_share_price_idx = pd.DataFrame(data = temp, columns=['Total_Price_Index'])
#df_total_share_price_idx = df_total_share_price_idx.rename_axis('Date').reset_index()
#print(df_total_share_price_idx.head(5))

# Combine FRED data and define real return for each month
#df_fred = pd.merge(df_total_share_price_idx, df_dividends, left_on='Date', right_on='Date')
#df_fred = pd.merge(df_fred, df_cpi, left_on='Date', right_on='Date')

#df_fred['Real_Price_Index'] = df_fred['Total_Price_Index']*df_fred['CPI_ADJ']
#df_fred['Real_Dividends_Index'] = df_fred['Total_Dividends_Index']*df_fred['CPI_ADJ']
#df_fred['Real_Price_Index_Lag'] = df_fred['Real_Price_Index'].shift()

#df_fred['Real_Month_Return'] = (df_fred['Real_Price_Index'] + df_fred['Real_Dividends_Index']) / df_fred['Real_Price_Index_Lag']
#df_fred['Real_Total_Return'] = df_fred['Real_Price_Index']
#df_fred['Real_Month_Return'].where(np.arange(len(df_fred))>0, 0).add(1).cumprod() * df_fred.loc[0, 'Real_Total_Return']
#print(df_fred)

# Read in data files 
# GOP term flags
url = "https://github.com/llorracc/as.180.369/blob/main/materials/presidential-term-data/GOP_flag.csv?raw=true"
temp = pd.read_csv(url,dtype=str)
df_gop = pd.DataFrame(temp)

# Add a specific day (e.g., the first day of the month)
df_gop['Date'] = df_gop['Date'] + '.01'

# Convert the complete date string to a datetime object
#df_gop['Date'] = datetime.strptime(df_gop['Date'], '%Y-%m-%d')
df_gop['Date'] = pd.to_datetime(df_gop['Date'])
df_gop['Date'] = df_gop['Date'].dt.normalize()
df_gop['Date'] = df_gop['Date'].dt.floor('D')
print(df_gop.head)

# Create new data frame that merges GOP flag with returns data
df = pd.merge(df_sentiment, df_gop, left_on='Date', right_on='Date')
print(df.head(5))


# Run OLS regression of consumer sentiment onto GOP president flag
# Outcome consumer sentiment next period. GOP president is pre-determined. 
df['GOPlag'] = df['GOP'].shift()

results = sm.formula.ols(formula = 'consumer_sentiment_index ~ GOPlag', data=df).fit(cov_type='HC3')
print(results.summary())

        Date  consumer_sentiment_index
0 1978-01-01                      83.7
1 1978-02-01                      84.3
2 1978-03-01                      78.8
3 1978-04-01                      81.6
4 1978-05-01                      82.9
<bound method NDFrame.head of            Date GOP
0    1871-01-01   1
1    1871-02-01   1
2    1871-03-01   1
3    1871-04-01   1
4    1871-05-01   1
...         ...  ..
1826 2023-03-01   0
1827 2023-04-01   0
1828 2023-05-01   0
1829 2023-06-01   0
1830 2023-07-01   0

[1831 rows x 2 columns]>
        Date  consumer_sentiment_index GOP
0 1978-01-01                      83.7   0
1 1978-01-01                      83.7   0
2 1978-02-01                      84.3   0
3 1978-03-01                      78.8   0
4 1978-04-01                      81.6   0
                               OLS Regression Results                               
Dep. Variable:     consumer_sentiment_index   R-squared:                       0.015
Model:                                  OL